In [1]:
import cv2
import torch
import numpy as np
import random
import os
import os.path

import utility
import data
import model
import loss
import template

from option_np import args
from trainer import Trainer

torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
if args.n_GPUs > 1:
    torch.cuda.manual_seed_all(args.seed) # if use multi-GPU
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
np.random.seed(args.seed)
random.seed(args.seed)

In [2]:
## Basic options and training options
args.n_GPUs = 4
args.model = 'DeblurUNet'
args.scale = '1'
args.rgb_range = 1
args.n_depth=2
args.n_resgroups = 4
args.n_resblocks = 16
args.data_train = 'REDS'
args.data_test = 'REDS_VAL'
args.data_range = '1-24000/1-100'
args.epochs = 400
args.loss = '1*L1+0.1*AE'
args.patch_size = 320
args.save = 'deblur_jpeg/wrcan/'
args.save_results = False
args.reset = False
args.ch_shuffle = True
args.lr_noise_sigma = 2
args.decay = '100-200-250-300-350'
args.weight_decay = 1e-8
args.precision = 'amp'
args.chop = True

## Continuing training options
# args.reset = False
# args.load = args.save
# args.resume = -1

## Test options
args.reset = False
args.data_range = '1-3000' #'1-3000'
args.test_only = True
args.pre_train = os.path.join('../experiment', args.save, 'model/model_best_patch320_epoch374.pt')
args.save_results = True
args.self_ensemble = False



template.set_template(args)

args.scale = list(map(lambda x: int(x), args.scale.split('+')))
args.dir_data = os.path.expanduser(args.dir_data)
args.dir_demo = os.path.expanduser(args.dir_demo)

args.data_train = args.data_train.split('+')
args.data_test = args.data_test.split('+')

if args.epochs == 0:
    args.epochs = 1e8

In [3]:
checkpoint = utility.checkpoint(args)


def main():
    global model
    if args.data_test == ['video']:
        from videotester import VideoTester
        model = model.Model(args, checkpoint)
        t = VideoTester(args, model, checkpoint)
        t.test()
    else:
        if checkpoint.ok:
            loader = data.Data(args)
            model = model.Model(args, checkpoint)
            loss_ = loss.Loss(args, checkpoint) if not args.test_only else None
            t = Trainer(args, loader, model, loss_, checkpoint)
            while not t.terminate():
                t.train()
                t.test()

            checkpoint.done()

if __name__ == '__main__':
    main()




Making model...
Preparing loss function:
1.000 * L1
0.100 * AE
[Epoch 1]	Learning rate: 1.00e-4
mode 1mode 1

mode 1mode 1

mode 0mode 0

mode 0
mode 0
mode 1mode 1

mode 1
mode 1


KeyboardInterrupt: 